In [1]:
#!pip install langgraph!pip3 -q install ipykernel jupyter
#!pip3  install torch torchvision torchaudio transformers
!pip3 install packaging ninja
!pip3 install accelerate
!pip3 install protobuf
!pip3 install sentencepiece
!pip3 install bitsandbytes
!pip3 install scipy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
#!pip install --upgrade langchain

In [3]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [10]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer, LlamaForCausalLM, MistralForCausalLM
import random

class Agent:
    def __init__(self, model,name):
        # Load Qwen model and tokenizer            
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,  # Changed from bfloat16 to float16
            bnb_4bit_quant_storage=torch.uint8,    # Added for storage optimization
            use_nested_quant=True,                 # Added for nested quantization
        )
        save_directory = model.replace('/','_')+'_saved'
        try:
            print("Trying to load from local:",save_directory)
            if 'Mistral' in save_directory:
                tokenizer = LlamaTokenizer.from_pretrained(
                  'teknium/OpenHermes-2.5-Mistral-7B',
                  trust_remote_code=True
                )

                model = MistralForCausalLM.from_pretrained(
                    "teknium/OpenHermes-2.5-Mistral-7B",
                    torch_dtype=torch.float16,
                    device_map="auto", #{'': 'cuda:0'},
                    load_in_8bit=False,
                    load_in_4bit=True,
                    use_flash_attention_2=False,
                    low_cpu_mem_usage=True
                )
            else:
                self.model = AutoModelForCausalLM.from_pretrained(
                   save_directory
                )
                self.tokenizer = AutoTokenizer.from_pretrained(save_directory)
        except:  
            print('The model:',model,'is not found locally, downloading it')
            self.model = AutoModelForCausalLM.from_pretrained(
                model, quantization_config=bnb_config
            )
            self.tokenizer = AutoTokenizer.from_pretrained(model)
            print("Saving the model:",model," locally")
            self.model.save_pretrained(save_directory)
            self.tokenizer.save_pretrained(save_directory)
        self.name = name
        self.model_name = model

    def generate_response(self, prompt):
        # Prepare input
        messages = [{"role": "user", "content": prompt}]
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Generate response
        inputs = self.tokenizer(text, return_tensors="pt", return_attention_mask=True).to(self.model.device)
        generated_ids = self.model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50
        )

        # Decode response
        response = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        return response


In [11]:
number_setter = None
guesser = None

In [12]:
class GuessingGame:
    def __init__(self, agent1, agent2,low_limit,high_limit):
        self.agent1 = agent1
        self.agent2 = agent2
        self.low_limit = low_limit
        self.high_limit = high_limit
        self.agent1.prompt = ''
        self.agent2.prompt = ''
        self.target = ''
        
    def clear_response(self,agent_name, response_string):
        #print('agent_name',agent_name)
        if all(keyword in agent_name for keyword in ['Qwen','Instruct']):
            #print('//////////////',response_string,'\n','//////////')
            return response_string.replace('ssistant.','%').split('ssistant\n')[1]
        if all(keyword in agent_name for keyword in ['falcon','instruct']): 
            return response_string.split('ssistant:')[1].split('User')[0] 
        
    def set_target_number(self):
        self.target_number = random.randint(self.low_limit, self.high_limit)
        print(f"{self.agent1.name} has set a secret number.{self.target_number}")
        
        return self.target_number

    def check_guess(self, guess):
        agent1_prompt =f"you know a secret number which is {self.target}.\
        if  {guess} is higher than {self.target}, reply: Try a lower number.\
        while if it is lower than < {self.target}, reply: Try a higher number, \
        Finally if it is equal to {self.target}, reply  'Correct!' \
        be breif in your reply"
        
        checking = self.agent1.generate_response(agent1_prompt)
        #print('agent1_prompt',agent1_prompt)
        #print('checking',checking)
        return checking

    def play_game(self):
        # Set target number
        self. target = self.set_target_number()
        guesses = []
        attempts = 0
        hint = ''
        init_prompt = f"you are keeping a secrete number which is {self.target}.\
        you already know this number. You will get responses that has new guessed number. just say guess higher or lower \
        number or just say it is equal. are you ready ?"
        agent_response = self.agent1.generate_response(init_prompt)
        agent1_response = self.clear_response(self.agent1.model_name,agent_response)
        print(f"initializing agent1: {agent1_response}")
        
        guess_prompt = f"Guess a number between {self.low_limit} and {self.high_limit}. Do not ask anything, just respond only with the number you guess."
        guess_response = self.agent2.generate_response(guess_prompt)
        guess_response = self.clear_response(self.agent2.model_name,guess_response)
        
        while attempts < 50:  # Limit attempts
           
            attempts += 1
            guesses.append(guess_response)
            
            print(f"{self.agent2.name} guesses: {guess_response}")
            
            # Check guess
            agent_response_loop = self.check_guess(guess_response)
            check_response =  self.clear_response(self.agent1.model_name,agent_response_loop)
            print(f"{self.agent1.name} says: {check_response}")
            # Check if correct
            if 'Correct ' in check_response or 'correct' in check_response:
                print ('agent1 completes',check_response)
                print(f"Game won in {attempts + 1} attempts!")
                break
            # Generate guess prompt
           
            # Get guess from other agent
            guess_prompt = f"Please,  {check_response}, and it should not be any number in this list {guesses}\
             and must be between {self.low_limit} and {self.high_limit}. \
            be breif in your reply and just respond only with the new number you guessed."
            guess_response = self.agent2.generate_response(guess_prompt)
            guess_response = self.clear_response(self.agent2.model_name,guess_response)
            
        else:
            print(f"Game over. The number was {self.target}")

# Note: This is a conceptual implementation
# Actual usage requires Qwen model installation and proper setup

In [13]:
def main():
    global number_setter, guesser
    READER_MODEL_NAME1 = "Qwen/Qwen2.5-Coder-7B-Instruct"
    READER_MODEL_NAME2 = "tiiuae/falcon-7b-instruct"
    READER_MODEL_NAME3 = 'teknium/OpenHermes-2.5-Mistral-7B'
    if number_setter is None:
        number_setter = Agent(READER_MODEL_NAME3, "NumberSetter")
        guesser = Agent(READER_MODEL_NAME3, "Guesser")
    playgame = GuessingGame(number_setter, guesser, 1,10)
    playgame.play_game()



In [14]:
if __name__ == "__main__":
    main()

Unused kwargs: ['use_nested_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Trying to load from local: teknium_OpenHermes-2.5-Mistral-7B_saved
The model: teknium/OpenHermes-2.5-Mistral-7B is not found locally, downloading it


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [10]:
from langchain.agents import AgentExecutor as TheAgent
asker_agent = TheAgent.create_chat_agent(
    model=model1,
    tokenizer=tokenizer1,
    name="Asker",
    #device='GPU',
    prompt_template="{history}\nAsker: {input}\n"
)

guesser_agent = TheAgent.create_chat_agent(
    model=model2,
    tokenizer=tokenizer2,
    name="Guesser",
    #device='GPU',
    prompt_template="{history}\nGuesser: {input}\n"
)

# Set up the conversation
conversation = Conversation([asker_agent, guesser_agent])

AttributeError: create_chat_agent

In [17]:
 import langchain.tools
dir(langchain.tools)

['Any',
 'BaseTool',
 'LangChainDeprecationWarning',
 'StructuredTool',
 'Tool',
 '_DEPRECATED_TOOLS',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_import_python_tool_PythonAstREPLTool',
 '_import_python_tool_PythonREPLTool',
 'is_interactive_env',
 'render',
 'tool',
 'warnings']

In [ ]:
main()

In [27]:
# Define two agents



TypeError: Can't instantiate abstract class Agent with abstract methods _get_default_output_parser, create_prompt, llm_prefix, observation_prefix

In [ ]:
import random
def guess_number_game(model1, tokenizer1, model2, tokenizer2):
    target_number = random.randint(1, 100)
    prompt0 = f"consider your self is playing a guessing play. You are player 1 who knows the number. \
    It is {target_number}. There will be player 2 who you will ask to guess the number. \
    Are you ready for the game player 1 ?"
    prompt1 = f"Ask player too to guess a number between 1 and 100"
    #prompt2 = "Guess a number between 1 and 100. without asking me any.. just say a number."
    response1 = generate_response(model1, tokenizer1, prompt0)
    print(f'Agent1: ',response1)
    return
    counter = 0 
    while True:
        # Agent 1's turn
        counter +=1
        response1 = generate_response(model1, tokenizer1, prompt1)
        print(f"Agent 1: {response1}")
        
        prompt2 = response1
        # Agent 2's turn
        response2 = generate_response(model2, tokenizer2, prompt2)
        #guess = int(response2.strip())
        print(f"Agent 2: {response2}")
        prompt1 = response2
        if counter > 10:
            break
         
        if 'It is equal' in response1:
            break
        '''
        else:
            continue
        if guess == target_number:
            print("Agent 2 guessed correctly!")
            break
        elif guess < target_number:
            prompt2 = "Your guess was too low. Try guessing a higher number."
        else:
            prompt2 = "Your guess was too high. Try guessing a lower number."
        '''

In [ ]:

# Start the game
guess_number_game(chatpot1, tokenizer1, chatpot2, tokenizer2)